In [1]:
import sys

import polars as pl
import numpy as np
from polang import polang
from pyparsing import *
from pyparsing import pyparsing_common as ppc

ModuleNotFoundError: No module named 'polang'

In [ ]:
# Trigonometric functions
expr = polang("alias(sin(a), 'sinus of a')")
print(expr)

df = pl.DataFrame({"Time": np.linspace(0,1), "a": np.linspace(1,2), "b": np.linspace(2,3), "c": np.linspace(3,4)})
df.select([pl.all(), expr])

col("a").sin().alias("sinus of a")


Time,a,b,c,sinus of a
f64,f64,f64,f64,f64
0.0,1.0,2.0,3.0,0.841471
0.020408,1.020408,2.020408,3.020408,0.852322
0.040816,1.040816,2.040816,3.040816,0.862817
0.061224,1.061224,2.061224,3.061224,0.872953
0.081633,1.081633,2.081633,3.081633,0.882726
0.102041,1.102041,2.102041,3.102041,0.892131
0.122449,1.122449,2.122449,3.122449,0.901165
0.142857,1.142857,2.142857,3.142857,0.909823
0.163265,1.163265,2.163265,3.163265,0.918102


In [2]:
# Derivative
expr = polang("shift(a, 1) - shift(a, -1)")
print(expr)

df = pl.DataFrame({"Time": np.linspace(0,1), "a": np.linspace(1,2), "b": np.linspace(2,3), "c": np.linspace(3,4)})
df.select([pl.all(), expr.alias("da/dt")])

NameError: name 'polang' is not defined

In [27]:
class Foo:
    
    def bar(self, *args):
        print(args)

foo = Foo()

In [37]:
def methodcaller(fname):
    def call(*args):
        return getattr(args[0], fname)(*args[1:])
    return call

methodcaller("bar")(foo, 1, 2, 3)

(1, 2, 3)


TypeError: method_getter.<locals>.method() missing 1 required positional argument: 'obj'